In [ ]:
from mia_models import train_target_model, load_shadow_data, train_shadow_models
from mia_models import prepare_attack_test_data, prep_validation_data, prep_attack_train_data
from mia_models import shokri_attack, prety_print_result
import tensorflow.keras as keras
import numpy as np
from sklearn.utils import resample
import pandas as pd
import pickle
import os
import csv
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.linear_model import LogisticRegression
from statistics import mean
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD
from keras.regularizers import l2
from sklearn.metrics import accuracy_score
MODEL_PATH = './model/'
DATA_PATH = './data/'

In [ ]:
# --------------------------------------------Original Data--------------------------------------------------------------#

In [ ]:
dataset_name = 'location'
train_size = 100
attack_test_size = 50
test_start = 100

loc_data = pd.read_csv('data/Location/bangkok', na_values=["?"], header=None)
target_dataset = loc_data.sample(n = 1200, replace = False)
df_rest = loc_data.loc[~loc_data.index.isin(target_dataset.index)]
shadow_dataset = df_rest.sample(n = 2000, replace = False)
df_rest = df_rest.loc[~df_rest.index.isin(shadow_dataset.index)]
attack_test_nonmembers = df_rest.sample(n = attack_test_size, replace = False)
attack_test_members =  target_dataset.iloc[:train_size,:].sample(n = attack_test_size, replace = False)

In [ ]:
# trian target model
per_class_sample=40
channel=1   
epochs=100
n_class = 31
is_synthetic = False
VERBOSE = 0
test_ratio = 0.3

target_model, dim = train_target_model(target_dataset, dataset_name, per_class_sample, epochs, n_class, train_size, test_start, is_synthetic)

In [ ]:
#train shadow model
n_shadow_models = 20
shadow_data_size = 1200

load_shadow_data(shadow_dataset, dataset_name, n_shadow_models, shadow_data_size, test_ratio, is_synthetic)
n_shadow_train_performance, n_shadow_test_performance, n_attack_data, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, shadow_model_init, shadow_accuracy = train_shadow_models(dataset_name, n_shadow_models, n_class, dim, epochs, channel)

In [ ]:
#train attack model
attack_test_data = prepare_attack_test_data(dataset_name, attack_test_members, attack_test_nonmembers, target_model, is_synthetic)
mem_validation, nmem_validation = prep_validation_data(attack_test_data)
attack_train_df = prep_attack_train_data(n_attack_data)
pred_membership, ori_membership, TP_idx_list, TN_idx_list, mpred, nmpred = shokri_attack(attack_train_df, mem_validation, nmem_validation, epochs)
tp, fp, fn, tn, precision, advj, acc, recall = prety_print_result (ori_membership,pred_membership)
print('Accuracy: ', acc, 'Precision: ', precision)

In [ ]:
mpredi = np.array([item for sublist in mpred for item in sublist])
len(mpredi)

In [ ]:
nmpredi = np.array([item for sublist in nmpred for item in sublist])

In [ ]:
df = pd.DataFrame(mpredi)
df['memberstatus'] = 'member'
df_nm = pd.DataFrame(nmpredi)
df_nm['memberstatus'] = 'nonmember'
df_nm
frames = [df, df_nm]
df_org = pd.concat(frames)
df_org

In [ ]:
# --------------------------------------------Original Data--------------------------------------------------------------#

In [ ]:
# --------------------------------------------Synthetic Data------------------------------------------------------------#

In [ ]:
# synthetic dataset
train_size = 100
attack_test_size = 50
test_start = 100
org_dataset = pd.read_csv('data/Location/bangkok', na_values=["?"], header=None)
target_dataset = pd.read_csv('data/Location/loc_sds_cac_200.csv', na_values=["?"], header=None).sample(n = 150, replace = False)
df = org_dataset.iloc[2500:,:]
# shadow_dataset = df.sample(n = 1900, replace = False)
# df_rest = df.loc[~df.index.isin(shadow_dataset.index)]
# attack_test_nonmembers = df_rest.sample(n = attack_test_size, replace = False)
# attack_test_members = org_dataset.iloc[:200,:]
# attack_test_members = attack_test_members.sample(n=attack_test_size, replace=False)

In [ ]:
# trian target model
per_class_sample=40
channel=1   
epochs=100
n_class = 31
is_synthetic = False
VERBOSE = 0
test_ratio = 0.3

target_model, dim = train_target_model(target_dataset, dataset_name, per_class_sample, epochs, n_class, train_size, test_start, is_synthetic)

In [ ]:
#train shadow model
n_shadow_models = 20
shadow_data_size = 1200

load_shadow_data(shadow_dataset, dataset_name, n_shadow_models, shadow_data_size, test_ratio, is_synthetic)
n_shadow_train_performance, n_shadow_test_performance, n_attack_data, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, shadow_model_init, shadow_accuracy = train_shadow_models(dataset_name, n_shadow_models, n_class, dim, epochs, channel)

In [ ]:
#train attack model
attack_test_data = prepare_attack_test_data(dataset_name, attack_test_members, attack_test_nonmembers, target_model, is_synthetic)
mem_validation, nmem_validation = prep_validation_data(attack_test_data)
attack_train_df = prep_attack_train_data(n_attack_data)
pred_membership, ori_membership, TP_idx_list, TN_idx_list, mpred_syn, nmpred_syn = shokri_attack(attack_train_df, mem_validation, nmem_validation, epochs)
tp, fp, fn, tn, precision, advj, acc, recall = prety_print_result (ori_membership,pred_membership)
print('Accuracy: ', acc, 'Precision: ', precision)

In [ ]:
mpredi_syn = np.array([item for sublist in mpred_syn for item in sublist])
nmpredi_syn = np.array([item for sublist in nmpred_syn for item in sublist])
mpredi_syn

In [ ]:
df_syn_m = pd.DataFrame(mpredi_syn)
df_syn_m['memberstatus'] = 'member'
df_syn_nm = pd.DataFrame(nmpredi_syn)
df_syn_nm['memberstatus'] = 'nonmember'
frames_syn = [df_syn_m, df_syn_nm]
df_syn = pd.concat(frames_syn)

In [ ]:
df_org['syn_0'] = df_syn[[0]]
df_org['syn_1'] = df_syn[[1]]
df_org['syn_mem_status'] = df_syn[['memberstatus']]
df_org

In [ ]:
df_org.to_csv('data/prediction_comparison_org_vs_syn.csv', index=False)

In [ ]:
# --------------------------------------------Synthetic Data------------------------------------------------------------#